In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_curve, auc, confusion_matrix, classification_report, accuracy_score, f1_score, matthews_corrcoef
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

df = pd.read_excel("results from fractal and DWT analyses.xlsx")
X = df[["Fractal Dimension", "Lacunarity", "WavHL Energy", "WavLH Energy", "WavHH Energy"]]
y = df["Class"]
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42)
rf = RandomForestClassifier(random_state=55)
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False]
}
cv_rf = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, scoring='accuracy')
cv_rf.fit(X_train, y_train)
best_rf = cv_rf.best_estimator_
y_pred = best_rf.predict(X_test)
y_prob = best_rf.predict_proba(X_test)[:, 1]
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Accuracy Score:", accuracy_score(y_test, y_pred))
print("F1:", f1_score(y_test, y_pred, pos_label="damaged"))
print("Matthews Correlation Coefficient:", matthews_corrcoef(y_test, y_pred))
fpr, tpr, thresholds = roc_curve(y_test, y_prob, pos_label="damaged")
roc_auc = auc(fpr, tpr)
plt.figure()
plt.plot(fpr, tpr, lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], lw=2, linestyle='-')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive')
plt.ylabel('True Positive ')
plt.title('ROC')
plt.legend(loc="lower right")
plt.show()
